In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('quora_duplicate_questions.csv')

In [3]:
df.sample(10)

,index,id,qid1,qid2,question1,question2,is_duplicate
253414,253414,297008,583401,583402,How do you format the title of a poem in an es...,What is the font used on the cover of the book...,0
4816,4816,5655,11302,11303,What is the best lucky number?,What is lucky number?,0
207898,207898,243609,479787,479788,How do I tell a girl that she is my crush?,How do I tell a girl I like her?,0
223061,223061,261407,514395,514396,What can be the eighth wonder of the world?,Which is the world's eighth wonder?,0
46262,46262,54230,107915,107916,What has been the effect of demonetisation on ...,What is the effect of demonetisation on your l...,1
196779,196779,230570,454348,454349,If Donald Trump stood in the middle of 5th Ave...,Is it true if Trump murdered someone people wo...,1
232464,232464,272475,535846,535847,What was the weirdest compliment you've ever r...,How can I give people the best compliment they...,0
61136,61136,71652,142402,142403,When is the best time to jump off a cliff?,How should you land in water if jumping from a...,0
186447,186447,218477,23545,430750,How should one prepare for a marathon?,How can I learn marathon?,1
288361,288361,337978,662578,662579,Why is Saltwater Taffy candy imported in Hong ...,Why is Saltwater taffy candy imported in South...,1


In [4]:
new_df = df.sample(30000)

In [5]:
new_df.isnull().sum()

index           0
id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [6]:
new_df.duplicated().sum()

0

In [7]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
237348,How does it feel to kill a human?,What is it like to kill someone?
301022,What's the difference between a check card and...,What's the deal with debit cards and credit ca...
103286,What is the most intuitive explanation for the...,What is an intuitive explanation of the chi-sq...
96254,What are the main reasons why students from th...,What are the main reasons why students from th...
233917,What is non load bearing wall?,How do you frame a load bearing wall?


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)


In [11]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [13]:
temp_df.head()


,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
237348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
301022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103286,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
233917,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [14]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [15]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
237348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
301022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103286,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
96254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
233917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7273333333333334

In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7131666666666666